In [1]:
from seagul.rl.models import PpoModel


import torch.nn as nn
from seagul.rl.algos.ppo import ppo
from seagul.nn import MLP, CategoricalMLP
import torch

import matplotlib.pyplot as plt
import seagul.envs

torch.set_default_dtype(torch.double)

input_size = 4
output_size = 1
layer_size = 12
num_layers = 2
activation = nn.ReLU

policy = MLP(input_size, output_size, num_layers, layer_size, activation)
value_fn = MLP(input_size, 1, num_layers, layer_size, activation)
model = PpoModel(policy, value_fn, action_var=.1, discrete=False)

# Define our hyper parameters
num_epochs = 100
batch_size = 2048  # how many steps we want to use before we update our gradients
num_steps = 1000  # number of steps in an episode (unless we terminate early)
max_reward = num_steps
p_batch_size = 1024
v_epochs = 1
p_epochs = 10
p_lr = 1e-2
v_lr = 1e-2

gamma = 0.99
lam = 0.99
eps = 0.2

# env2, t_policy, t_val, rewards = ppo('InvertedPendulum-v2', 100, policy, value_fn)
t_model, rewards, var_dict = ppo("sg_cartpole-v0", 100, model, action_var_schedule=[1])
print(rewards)
plt.plot(rewards)
plt.show()


100%|██████████| 100/100 [02:54<00:00,  1.76s/it]

[130.375, 153.4, 115.94444444444444, 140.06666666666666, 132.375, 132.0625, 124.0, 130.5, 158.46153846153845, 132.5625, 121.41176470588235, 140.13333333333333, 120.52941176470588, 151.92857142857142, 121.23529411764706, 130.4375, 163.3846153846154, 127.5, 147.64285714285714, 123.82352941176471, 140.46666666666667, 129.4375, 123.70588235294117, 138.53333333333333, 156.14285714285714, 113.73684210526316, 123.94117647058823, 127.75, 116.55555555555556, 210.3, 141.26666666666668, 156.85714285714286, 177.75, 137.13333333333333, 116.26315789473684, 144.0, 160.5, 153.14285714285714, 150.64285714285714, 113.5, 134.125, 147.6, 155.64285714285714, 121.16666666666667, 124.23529411764706, 149.57142857142858, 131.5625, 140.06666666666666, 137.73333333333332, 137.86666666666667, 128.9375, 144.26666666666668, 163.15384615384616, 150.5, 140.46666666666667, 160.0, 145.64285714285714, 131.375, 121.52941176470588, 134.4375, 150.21428571428572, 121.44444444444444, 211.6, 150.2, 138.3125, 144.2666666666666

<Figure size 640x480 with 1 Axes>